In [ ]:
# intent_classifier.ipynb
# Asumimos que las variables de data_processing.ipynb (df, preprocess_text) y imports.ipynb (TfidfVectorizer, MultinomialNB) están disponibles.

intent_classifier = None
tfidf_vectorizer = None

if df is not None:
    from sklearn.model_selection import train_test_split

    X = df['Consulta_Processed'] # Texto de la consulta
    y = df['Tipo_Consulta']     # Intención (categoría)

    # Dividir en conjuntos de entrenamiento y prueba (necesario para entrenar)
    # En producción, solo cargarías el modelo entrenado
    # Verificar si hay suficientes muestras por clase para estratificar
    if y.nunique() > 1 and min(y.value_counts()) >= 2:
         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    else:
        print("Advertencia: No hay suficientes muestras por clase para estratificar. Dividiendo sin estratificación.")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # Vectorización de texto (TF-IDF)
    # En producción, solo cargarías el vectorizador entrenado
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    # X_test_tfidf = tfidf_vectorizer.transform(X_test) # No necesitas X_test_tfidf para la predicción en producción

    # Entrenamiento del modelo
    intent_classifier = MultinomialNB() # O LinearSVC()
    intent_classifier.fit(X_train_tfidf, y_train)

    print("Clasificador de intenciones entrenado (o listo para cargar).")
else:
    print("No se pudo entrenar el clasificador debido a un error de carga de datos.")

# Función para clasificar una nueva consulta
def classify_intent(query):
    if intent_classifier is None or tfidf_vectorizer is None:
        return "Error: Modelo de clasificación no cargado o entrenado."
    processed_query = preprocess_text(query)
    query_tfidf = tfidf_vectorizer.transform([processed_query])
    prediction = intent_classifier.predict(query_tfidf)[0]
    return prediction

# Ejemplo de uso (para probar este notebook individualmente)
# if df is not None:
#     test_query = "¿Cuándo son las matrículas?"
#     predicted_intent = classify_intent(test_query)
#     print(f"\nTest Query: '{test_query}' -> Predicted Intent: {predicted_intent}")